<a href="https://colab.research.google.com/github/darrenpierre90/Machine-Learning-with-Linear-classifers/blob/master/COMP4900A1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# COMP 4900 Assignment 1

*   Tess Landry: 100926518
*   Darren Pierre: 101015833



# Initial Set-up
 * The followling block code is to setup our project 


In [0]:
from google.colab import drive
drive.mount('/content/drive')
PARKINSON_DATA="/content/drive/My Drive/Colab Notebooks/Data Sets/parkinsons.data"
SONAR_DATA="/content/drive/My Drive/Colab Notebooks/Data Sets/sonar.all-data"


Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


# Start of the Project

In [0]:
# Importing libaries that we need for the project 
import os 
import pandas as pd 
import numpy as np 
import sklearn.linear_model
!pip install ipython-autotime
%load_ext autotime



  Created wheel for ipython-autotime: filename=ipython_autotime-0.1-cp36-none-any.whl size=1832 sha256=d9d93057e92c66b0c0530abfbbfe36622c4fff18b9457d8460df326de9df8c2b
  Stored in directory: /root/.cache/pip/wheels/d2/df/81/2db1e54bc91002cec40334629bc39cfa86dff540b304ebcd6e
Successfully built ipython-autotime


# Parkinsons Disease Data Set

## Data Set Information:

This dataset is composed of a range of biomedical voice measurements from 
31 people, 23 with Parkinson's disease (PD). Each column in the table is a 
particular voice measure, and each row corresponds one of 195 voice 
recording from these individuals ("name" column). The main aim of the data 
is to discriminate healthy people from those with PD, according to "status" 
column which is set to 0 for healthy and 1 for PD.

 The rows of the CSV file contain an 
instance corresponding to one voice recording. There are around six 
recordings per patient, the name of the patient is identified in the first 
column.


## Attribute Information:

### Matrix column entries (attributes):

| Attribute Name | Description |
| ------------------------------ | ------------- |
|name | ASCII subject name and recording number |
|MDVP:Fo(Hz) | Average vocal fundamental frequency |
|MDVP:Fhi(Hz) | Maximum vocal fundamental frequency |
|MDVP:Flo(Hz) | Minimum vocal fundamental frequency |
|MDVP:Jitter(%),MDVP:Jitter(Abs),MDVP:RAP,MDVP:PPQ,Jitter:DDP | Several measures of variation in fundamental frequency |
|MDVP:Shimmer,MDVP:Shimmer(dB),Shimmer:APQ3,Shimmer:APQ5,MDVP:APQ,Shimmer:DDA | Several measures of variation in amplitude |
|NHR,HNR | Two measures of ratio of noise to tonal components in the voice |
|status | Health status of the subject (one)  |
|RPDE,D2 | Two nonlinear dynamical complexity measures |
|DFA | Signal fractal scaling exponent |
|spread1,spread2,PPE | Three nonlinear measures of fundamental frequency variation  |



 # Sonar, Mines vs. Rocks Data Set

## SUMMARY: 
This is the data set used by Gorman and Sejnowski in their study
of the classification of sonar signals using a neural network [1].  The
task is to train a network to discriminate between sonar signals bounced
off a metal cylinder and those bounced off a roughly cylindrical rock.


## PROBLEM DESCRIPTION:

The file "sonar.mines" contains 111 patterns obtained by bouncing sonar
signals off a metal cylinder at various angles and under various
conditions.  The file "sonar.rocks" contains 97 patterns obtained from
rocks under similar conditions.  The transmitted sonar signal is a
frequency-modulated chirp, rising in frequency.  The data set contains
signals obtained from a variety of different aspect angles, spanning 90
degrees for the cylinder and 180 degrees for the rock.

Each pattern is a set of 60 numbers in the range 0.0 to 1.0.  Each number
represents the energy within a particular frequency band, integrated over
a certain period of time.  The integration aperture for higher frequencies
occur later in time, since these frequencies are transmitted later during
the chirp.

The label associated with each record contains the letter "R" if the object
is a rock and "M" if it is a mine (metal cylinder).  The numbers in the
labels are in increasing order of aspect angle, but they do not encode the
angle directly.





# Dataset Set-Up, Formatting, and Feature Selection

In [0]:

parkinsonDF=pd.read_csv(PARKINSON_DATA)
sonarDF=pd.read_csv(SONAR_DATA)
numCols=61
names=[f"Feature {x}"for x in range(numCols)]
# Class we will try to classify  will be a mine 
sonarDF=pd.read_csv(SONAR_DATA,header=None , names=names)
convertValues=lambda a :1 if a == 'M' else 0
sonarDF[names[-1]]=[convertValues(label) for label in sonarDF[names[-1]]]

#GET RID OF UNWANTED PARKINSONS COLUMNS
parkinsonDF = parkinsonDF.drop('name', 1)


"""
FEATURE SELECTION FOR PARKINSONS
L1 (Lasso) Regression: Choose features not weighted 0
"""
parkinsonX = parkinsonDF.drop('status', 1)
parkinsonY = parkinsonDF['status']
parkCols = parkinsonX.columns.to_list()

parkL1 = sklearn.linear_model.Lasso(alpha=.001) #You can play around with this to get more or less features
parkL1.fit(parkinsonX, parkinsonY)

#Print all items that arent weighted 0- these are the selected features
print("Parkinsons Selected Features: \n")
parkFeats = [] #List for the selected features
i = 0
while i < len(parkL1.coef_):
  if(parkL1.coef_[i] != 0):
    print(str(parkCols[i]) + ": " + str(parkL1.coef_[i]))
    parkFeats.append(parkCols[i])
  i = i+1
print("\n\n")
parkFeats.append('status')

#Make a df of just the selected features:
featSelectPark = parkinsonDF[parkFeats]


"""
FEATURE SELECTION FOR SONAR
L1 (Lasso) Regression: Choose features not weighted 0
"""
sonarX = sonarDF.drop('Feature 60', 1)
sonarY = sonarDF['Feature 60']
sonarCols = sonarX.columns.to_list()

sonarL1 = sklearn.linear_model.Lasso(alpha=0.01) #Basic highest alpha to not give all 0s
sonarL1.fit(sonarX, sonarY)

#Print all items that arent weighted 0- these are the selected features
print("Sonar Selected Features: \n")
sonarFeats = [] #List for the selected features
i = 0
while i < len(sonarL1.coef_):
  if(sonarL1.coef_[i] != 0):
    print(str(sonarCols[i]) + ": " + str(sonarL1.coef_[i]))
    sonarFeats.append(sonarCols[i])
  i = i+1
print("\n")
sonarFeats.append('Feature 60')

#Make a df of just the selected features:
featSelectSonar = sonarDF[sonarFeats]

print('All features not shown above had weight = 0 and were not selected.')

"""
A class that takes in a pandas dataset and returns the X Matrix and Y Vector as numpy arrays
Input: 
    dataSet: pandas dataSet to test (get rid of any unwanted columns BEFORE doing this)
    yName: column name of output (Y) variable
NOTE: Format data set BEFORE passing in (eg. remove columns you dont want such as 'name')
"""
class dataSetFormatter:
  #Returns the numpy XMatrix
  def xMatrix(self, dataSet, yName):
    return (dataSet.drop(yName, 1)).to_numpy() 

  def yMatrix(self, dataSet, yName):
    return dataSet[[yName]].to_numpy()


#MAKE A FORMATTER
myFormatter = dataSetFormatter() 


Parkinsons Selected Features: 

MDVP:Fo(Hz): -0.0016386952741853078
MDVP:Fhi(Hz): -0.00043834867783684386
MDVP:Flo(Hz): -0.0013258927998608815
MDVP:Shimmer(dB): 0.003135982169228187
NHR: -0.42072729925921815
HNR: -0.002240062912846496
RPDE: -0.29674350854948284
DFA: 0.14501093699328377
spread1: 0.13805729578668324
spread2: 0.33794501099757485
D2: 0.20569318197081668



Sonar Selected Features: 

Feature 10: 0.5561138224634387
Feature 11: 0.1608526778546402
Feature 16: -0.009235214574464422
Feature 20: 0.16215826781486622
Feature 35: -0.37028715196517226
Feature 44: 0.7367015306618203


All features not shown above had weight = 0 and were not selected.
time: 110 ms


# Implementing Machine Learning Models

In [0]:
"""
A class to represent the performance of a Machine learning model 
Input: Either a list or numpy arrays representing the predicted values and the actual values 
"""
class MLMetric:
  def __init__(self,predictedValues,outputValues):
    self.predictedValues=predictedValues
    self.outputValues=outputValues
  
  def percentage(self,num):
    return format(num,".2%")
  #Evaluates the accuracy of the model, comparing predicted values with actual output values
  def Accu_eval(self):
    correctAns=0
    i = 0
    while i < len(self.predictedValues):
      if (self.predictedValues[i] == self.outputValues[i]):
        correctAns=correctAns +1
      i=i+1
    
    accuracy=correctAns/len(self.predictedValues)
    print(f"The model had {correctAns} correct out of {len(self.predictedValues)}. Resulted in accuracy of: {self.percentage(accuracy)} ")
    self.confusionMatrix() #Prints the confusion matrix data too
    return accuracy

  #Finds the confusion metrics
  def cMetrics(self):
    tp=fp=tn=fn=0 # values to represent true positive , false positive , true negative and false negative

    i = 0
    while i < len(self.predictedValues):
      if(self.outputValues[i]==1):
        if(self.predictedValues[i]==1):
          tp=tp+1
        else:
          fn=fn+1
      else:
        if(self.predictedValues[i]==1):
          fp=fp+1
        else:
          tn=tn+1
      i=i+1

    return [tp,fp,tn,fn]
  
  #Prints the confusion metrics nicely
  def confusionMatrix(self):
    confusionData = self.cMetrics()
    # True postive rate : When a model is given a 1 (a class that we're trying to classify) , how often it predicts yes (1)
    if((confusionData[0] + confusionData[3]) == 0): #Prevent division by 0
      TPR = 0
    else:
      TPR= confusionData[0] /(confusionData[0] + confusionData[3]) # TPR= TP/(TP + FN)
    # True negative Rate : When a model is given a 0 (a class that we're trying to classify) , how often it predicts no (0)
    if((confusionData[2] + confusionData[1]) == 0):
      TNR = 0
    else:
      TNR= confusionData[2] /(confusionData[2] + confusionData[1]) # TNR= TN/(TN + FP)
    print("\nConfusion Matrix:")
    print("   True Positives: " + str(confusionData[0]))
    print("   False Positives: " + str(confusionData[1]))
    print("   True Negatives: " + str(confusionData[2]))
    print("   False Negatives: " + str(confusionData[3]))
    print(f"  True Postive Rate: {self.percentage(TPR)}")
    print(f"  True Negative Rate: {self.percentage(TNR)}")
    print("\n")


time: 49.3 ms


In [0]:
"""
A class that represents a Logistic Regression Model
Input: 
    dataX: numpy array with that represent a training set without the classicfications 
    dataX: numpy array with that represent a training set 
    learning_rate: The amount that the weights are updated during training
    training_iterations: number of times we should train our model 
    y_intercept : our w0 term

"""
class LogisticRegression():
  def __init__(self,dataX,dataY,learning_rate,training_iteration,y_intercept=0):
    numFeatures=dataX.shape[1] # getting the number of features
    self.weights= np.zeros(numFeatures,dtype=np.float128) # setting our weights to zero
    self.X=dataX
    self.Y=dataY
    self.y_intercept=y_intercept
    self.fit(learning_rate,training_iteration)

  def sigmoid(self,x):
    # 1/ 1 + e ^-x     where x =wTx + y_intercept
    return 1 / (1 + np.exp(-x,dtype=np.float128))

  def computeSigmoidInput(self,xi):
    # wTx + y_intercept
    return np.dot(self.weights.T,xi) + self.y_intercept

    
  def fit(self,learning_rate,training_iteration):
        for _ in range(training_iteration):
          gradient=0
          numInstances=self.X.shape[0]
          for i in range(numInstances):
            xi=np.array(self.X[i],dtype=np.float128 )# xi represents an instance in our dataset 
            yi=np.array(self.Y[i],dtype=np.float128) # yi represents a label in our dataset    
            sigmoidInput= self.computeSigmoidInput(xi)# z= (wT * xi) + y-intercept
            predicted_output= self.sigmoid(sigmoidInput) # sigmoid(z)
            error = yi-predicted_output # error=y- sigmoid(z)
            gradient = gradient + (xi*error) # SUM(xi * error)

          # here we update our weights 
          self.weights = self.weights + (learning_rate * (gradient ))
   
  def predict(self,data):
            #Classification of a new observation , 
            output=[self.sigmoid(self.computeSigmoidInput(row)) for row in data] #sigmoid(wT * x_new + intercept)  
            classify= lambda x: 1 if x >= 0.5 else 0 
            predictedClasses=[classify(x) for x in output ]
            return predictedClasses

time: 33.8 ms


In [0]:
"""
A class to perform a Linear Discriminant Analysis
Input: 
    dataSetX: numpy array of X values
    dataSetY: numpy array of Y values such that the result for x_i is y_i
"""

class LDA:
  def __init__(self, dataSetX, dataSetY):

    self.X = dataSetX
    self.Y = dataSetY

    self.N_0 = 0  #Number of entries in class 0
    self.N_1 = 0  #Number of entries in class 1
    self.total = 0 #Total Entries

    self.pY0 = 0  #P(Y=0)
    self.pY1 = 0  #P(Y=1)

    self.mean0 = []  #Mean of class 0
    self.mean1 = []  #Mean of class 1 

    self.covariance = []  #Covariance Matrix
    self.covarInverse = [] #Inverse Covariance Matrix
    self.predictions = [] #Vector of Predictions for testData
    
    self.fit()
    #print("Class 0: " + str(self.N_0))
    #print("Class 1: " + str(self.N_1))

  def fit(self):
    #PROBABILITIES
    i=0
    while i < len(self.Y):
      if (self.Y[i] == 0):
        self.N_0 = self.N_0 + 1
      elif (self.Y[i] == 1):
        self.N_1 = self.N_1 + 1
      i=i+1

    self.total = len(self.Y) #len(self.Y) should be equal to N_0 + N_1, if not we have a problem
    self.pY0 = self.N_0 / self.total #P(y = 0) = N_0 / (N_0 + N_1) eg. if 20/50 samples are class 0
    self.pY1 = self.N_1 / self.total #P(y = 1) = N_1 / (N_0 + N_1) eg. if 30/50 samples are class 1

    #MEANS
    i=0
    while i < len(self.Y):
      if(self.Y[i] == 0):
        if(len(self.mean0) == 0):
          self.mean0 = self.X[i]
        else:
          self.mean0 = np.add(self.mean0, self.X[i])
      elif(self.Y[i] == 1):
        if(len(self.mean1) == 0):
          self.mean1 = self.X[i]
        else:
          self.mean1 = np.add(self.mean1, self.X[i])
      i=i+1
    
    self.mean0 = (np.true_divide(self.mean0, self.N_0)).reshape(-1,1) # μ_0 = (sum of i = 1 to i = n): I(y_i = 0) x_i/N_0, reshape so mx1
    self.mean1 = (np.true_divide(self.mean1, self.N_1)).reshape(-1,1) # μ_1 = (sum of i = 1 to i = n): I(y_i = 0) x_i/N_1, reshape so mx1

    #COVARIANCE MATRIX
    i=0
    while i < len(self.Y):
      currentX = (self.X[i]).reshape(-1,1) #Reshape so that it is mx1

      if(self.Y[i] == 0):
        xMinusMean = np.subtract(currentX, self.mean0)
      elif(self.Y[i] == 1):
        xMinusMean = np.subtract(currentX, self.mean1)

      covarNumerator = np.dot(xMinusMean, (xMinusMean.transpose()))
      covarDenominator = np.subtract(self.total, 2)
      covarTotal = np.true_divide(covarNumerator, covarDenominator)
      #on the first iteration, set it to first calculated matrix
      if(i == 0):
        self.covariance = covarTotal
      else:
        self.covariance = np.add(self.covariance, covarTotal)
      i=i+1 
    self.covarInverse = np.linalg.inv(self.covariance)
    #print(self.covariance)

  #Predict Function, takes data, predicts using the fit we calculated
  def predict(self, testData):
    i=0
    while i < len(testData):
      #This is all just that big formula line by line
      currData = testData[i].reshape(-1,1) #reshape so mx1
      currData = currData.transpose()
      currData = np.dot(currData, self.covarInverse)

 
      #Class 0
      part1_0 = np.dot(currData, self.mean0)
      part2_0 = self.mean0.transpose()
      part2_0 = np.multiply(part2_0, 0.5)
      part2_0 = np.dot(part2_0, self.covarInverse)
      part2_0 = np.dot(part2_0, self.mean0)
      part3_0 = np.log(self.pY0)

      discrim0 = np.subtract(part1_0, part2_0)
      discrim0 = np.add(discrim0, part3_0)

      #Class 1
      part1_1 = np.dot(currData, self.mean1)
      part2_1 = self.mean1.transpose()
      part2_1 = np.multiply(part2_1, 0.5)
      part2_1 = np.dot(part2_1, self.covarInverse)
      part2_1 = np.dot(part2_1, self.mean1)
      part3_1 = np.log(self.pY1)

      discrim1 = np.subtract(part1_1, part2_1)
      discrim1 = np.add(discrim1, part3_1)

      #Output ArgMax of Class 0 and Class 1
      if(discrim1 > discrim0):
        if (len(self.predictions) == 0):
          self.predictions = np.array([1])
        else:
          self.predictions = np.append(self.predictions, 1)
      else:
        if (len(self.predictions) == 0):
          self.predictions = np.array([0])
        else:
          self.predictions = np.append(self.predictions, 0)

      i=i+1
    self.predictions = self.predictions.reshape(-1,1)

    return self.predictions



time: 166 ms


In [0]:
"""
KFold class that takes in a pandas dataset to perform the fold
Input: 
    dataSet: pandas dataSet to test (get rid of any unwanted columns BEFORE doing this)
    k: number of folds to perform
    yName: name of y column, to be used when splitting data
"""

class KFoldValidation:
  def __init__(self,dataSet,k, yName):
    self.kSplitData = self.splitData(dataSet,k) #Data evenly (as much as possible) split into k partitions
    if(k < 2):
      self.k=2
    else:
      self.k=k
    self.yName = yName
    self.kFoldSets = [] #Set of k [trainSet, valSet] pairs. Perform LDA and Logistic regression on each one.
    
    #LDA SPECIFIC VARIABLES
    self.kFoldLDAs = [] #List of LDA Fits generated from the analysis. call kFoldLDAs[i].predict() to test any of your data on any of these
    self.accuracyLDA = 0

    #LR SPECIFIC VARIABLES
    self.kFoldLogs = list() #List of LR Fits generated from the analysis. call kFoldLogs[i].predict() to test any of your data on any of these
    self.accuracyLog = 0

    #Call these functions on init so that we already have our dataSets and LDA fits ready to go
    self.kFoldDataSet() 
    self.kFold_LDA_fit()
    


  def splitData(self, dataSet, k):
    dataSet = dataSet.sample(frac=1).reset_index(drop=True)
    kSplitList = []
  
    # checking to see how much is remaining if we divide our dataset by k 
    remainder= len(dataSet) % k

    #CREATE LIST OF k PARTITIONS OF THE DATA
    #NO REMAINDER
    if(remainder==0):
      i = 0
      start = 0
      sizeOfPartitions = int(len(dataSet) / k)
      stop = sizeOfPartitions
      while i < k:
        partition = dataSet[start:stop]
        kSplitList.append(partition)
        start = start + sizeOfPartitions
        stop = stop + sizeOfPartitions
        i=i+1

    #REMAINDER
    else:
      i = 0
      start = 0
      sizeOfPartitions = int(len(dataSet) // k)
      stop = sizeOfPartitions + 1
      while i < k:
        partition = dataSet[start:stop]
        kSplitList.append(partition)
        if(remainder > 0):             #REMAINDER > 0 means we have partition size + 1 partitions
          start = start + sizeOfPartitions + 1 #This round had partition + remainder so we MUST increment start by this much
          remainder = remainder - 1
          if(remainder > 0):           #If still > 0, we want next next round to include remainder
            stop = stop + sizeOfPartitions + 1        
          else:                        #otherwise this was our last iteration with the remainder and next round should have regular partitions
            stop = stop + sizeOfPartitions
        else:                          #REMAINDER = 0 means we have normal sized partitions
          start = start + sizeOfPartitions
          stop = stop + sizeOfPartitions
        i=i+1

    return kSplitList


  #Prepares a single dataSet consisting of [training set, validation set]
  def prepareDataSet(self,indexOfValSet):
    valSet = self.kSplitData[indexOfValSet].reset_index(drop=True)
    trainSet = []
    #Combine all the other data into our training set
    i = 0
    while i < self.k:
      if(i == indexOfValSet):
        i = i+1
        continue
      if(len(trainSet) == 0): #no entries added
        trainSet = self.kSplitData[i]
      else: #we concatenate
        trainSet = pd.concat([trainSet, self.kSplitData[i]], axis=0).reset_index(drop=True)
      i = i+1
  
    return [trainSet, valSet]

  #Prepares a k-length list of all [training set, validation set] pairs for the kfold
  def kFoldDataSet(self):
    i=0
    while i < self.k:
      self.kFoldSets.append(self.prepareDataSet(i))
      i=i+1
    #self.kFoldSets[i][0] is ith training set
    #self.kFoldSets[i][1] is ith validation set


  def kFold_LDA_fit(self):
    i = 0
    while i < self.k:
      #Use LDA to fit with the Training Set
      myLDA = LDA(myFormatter.xMatrix(self.kFoldSets[i][0], self.yName), myFormatter.yMatrix(self.kFoldSets[i][0], self.yName))
      #Add our fit to the list so that it can be tested with Test Data Later
      self.kFoldLDAs.append(myLDA)
      i=i+1

  def kFold_LDA_predict(self):
    predictionAccuracy = 0
    i = 0
    while i < self.k:
      #Use our fit to predict with the Validation Set
      myLDAPredictions = self.kFoldLDAs[i].predict(myFormatter.xMatrix(self.kFoldSets[i][1], self.yName))
      
      #Accuracy Metrics on Parkinsons
      myMetric = MLMetric(myLDAPredictions, myFormatter.yMatrix(self.kFoldSets[i][1], self.yName))
      predictionAccuracy = predictionAccuracy + myMetric.Accu_eval()
      i = i+1

    self.accuracyLDA = (predictionAccuracy/self.k)
    print("LDA Average Accuracy for this fit is: " + str(format(self.accuracyLDA,".2%")))

    return self.accuracyLDA

  def kFold_Log_fit(self,lr,itterations):
    i = 0
    while i < self.k:
      
      myLog = LogisticRegression(myFormatter.xMatrix(self.kFoldSets[i][0], self.yName), myFormatter.yMatrix(self.kFoldSets[i][0], self.yName),lr,itterations)
      #Add our fit to the list so that it can be tested with Test Data Later
      self.kFoldLogs.append(myLog)
      i=i+1 

  def kFold_Log_predict(self):
    predictionAccuracy = 0
    i = 0
    while i < self.k:
      #Use our fit to predict with the Validation Set

      
      myLDAPredictions = self.kFoldLogs[i].predict(myFormatter.xMatrix(self.kFoldSets[i][1], self.yName))
      
      #Accuracy Metrics on Parkinsons
      myMetric = MLMetric(myLDAPredictions, myFormatter.yMatrix(self.kFoldSets[i][1], self.yName))
      print("Summary for Logistic Regression:")
      predictionAccuracy = predictionAccuracy + myMetric.Accu_eval()
      i = i+1

    self.accuracyLog = (predictionAccuracy/self.k)
    print("Logistic Regression Average Accuracy for this fit is: " + str(format(self.accuracyLog,".2%")))

    return self.accuracyLog





time: 204 ms


# Getting Learning Rate for Logistic Regression




In [0]:
from matplotlib import pyplot as plt
def lineGraph(x,y,xlabel,title):
  labels =["Parkinson Data Set","Sonar vs Mines Dataset"]
  f, ax = plt.subplots()
  ax.plot(x, y[0] ,linestyle='-', label=labels[0],marker='o')
  ax.plot(x, y[1] ,linestyle='-', label=labels[1],marker='o')

  ax.set_xlabel(xlabel)
  ax.set_ylabel("Accuracy of the model")
  ax.set_title(title)
  plt.legend()
  plt.show()


def testRun(learningRate,itterations=1400):
  parkinsonsKFold = KFoldValidation(parkinsonDF, 10, 'status')
  parkinsonsKFold.kFold_Log_fit(learningRate,itterations)
  parkAvgAccuracy=parkinsonsKFold.kFold_Log_predict()
  sonarKFold = KFoldValidation(sonarDF, 10, 'Feature 60')
  sonarKFold.kFold_Log_fit(learningRate,itterations)
  sonarAvgAccuracy = sonarKFold.kFold_Log_predict()
  return parkAvgAccuracy,sonarAvgAccuracy

def learningRateTest(learningRates):
  graphTitle="Learning Rate effect on Logistic Regression performance"
  xLabel="Learning Rate"
  itterations=1400
  pScores=list()
  sScores=list()
  for lr in learningRates:
    parkinsonAccuracy,sonarAccuracy=testRun(lr,itterations)
    pScores.append(parkinsonAccuracy)
    sScores.append(sonarAccuracy)
  lineGraph(learningRates,(pScores,sScores),xLabel,graphTitle)
learningRates=[0.001,0.01,0.1]
learningRates2=[x* 0.0001 for x in range (1,11) ]
learningRates3=[x* 0.005 for x in range(1,6) ]
#learningRateTest(learningRates)


def itterationsTest(itterations):
  graphTitle="Number of itterations effect on Logistic Regression performance"
  xLabel="Number of itterations"
  learningRate=0.005
  pScores=list()
  sScores=list()
  for i in itterations:
    parkinsonAccuracy,sonarAccuracy=testRun(learningRate,i)
    pScores.append(parkinsonAccuracy)
    sScores.append(sonarAccuracy)
  lineGraph(itterations,(pScores,sScores),xLabel,graphTitle)
  print(f"Parkinsion Average KFold Scores:{pScores}") 
  print(f"Scores Average KFold Scores:{sScores}") 


itterations=[100,250,500,900,1400]

accuracyTest=[500 for x in range(5)]

time: 55.6 ms


# Running Our Algorithms

In [0]:
"""
LDA Run of PARKINSONS
"""

print("---------------")
print("PARKINSONS DATA")
print("---------------\n")

#ALL FEATURES
parkAllAcc = []
i=0
while i<5:
  parkinsonsKFold = KFoldValidation(parkinsonDF, 10, 'status') #K fold with all features
  parkinsonsAvgAcc = parkinsonsKFold.kFold_LDA_predict()
  parkAllAcc.append(format(parkinsonsAvgAcc,".2%"))
  i=i+1

#SELECT FEATURES
parkSelectAcc = []
i=0
while i<5:
  parkinsonsKFoldFeat = KFoldValidation(featSelectPark, 10, 'status') #K fold with selected features
  parkinsonsFeatAvgAcc = parkinsonsKFoldFeat.kFold_LDA_predict()
  parkSelectAcc.append(format(parkinsonsFeatAvgAcc, ".2%"))
  i=i+1

---------------
PARKINSONS DATA
---------------

The model had 17 correct out of 20. Resulted in accuracy of: 85.00% 

Confusion Matrix:
   True Positives: 15
   False Positives: 3
   True Negatives: 2
   False Negatives: 0
  True Postive Rate: 100.00%
  True Negative Rate: 40.00%


The model had 17 correct out of 20. Resulted in accuracy of: 85.00% 

Confusion Matrix:
   True Positives: 15
   False Positives: 2
   True Negatives: 2
   False Negatives: 1
  True Postive Rate: 93.75%
  True Negative Rate: 50.00%


The model had 18 correct out of 20. Resulted in accuracy of: 90.00% 

Confusion Matrix:
   True Positives: 15
   False Positives: 2
   True Negatives: 3
   False Negatives: 0
  True Postive Rate: 100.00%
  True Negative Rate: 60.00%


The model had 17 correct out of 20. Resulted in accuracy of: 85.00% 

Confusion Matrix:
   True Positives: 14
   False Positives: 3
   True Negatives: 3
   False Negatives: 0
  True Postive Rate: 100.00%
  True Negative Rate: 50.00%


The model ha

In [0]:
"""
LDA Run of SONAR
"""

print("\n\n----------")
print("SONAR DATA")
print("----------\n")

sonarAllAcc = []
i=0
while i<5:
  sonarKFold = KFoldValidation(sonarDF, 10, 'Feature 60') #K fold with all features
  sonarAvgAcc = sonarKFold.kFold_LDA_predict()
  sonarAllAcc.append(format(sonarAvgAcc, ".2%"))
  i=i+1

sonarSelectAcc = []
i=0
while i<5:
  sonarKFoldFeat = KFoldValidation(featSelectSonar, 10, 'Feature 60') #K fold with selected features
  sonarFeatAvgAcc = sonarKFoldFeat.kFold_LDA_predict()
  sonarSelectAcc.append(format(sonarFeatAvgAcc, ".2%"))
  i=i+1




----------
SONAR DATA
----------

The model had 14 correct out of 21. Resulted in accuracy of: 66.67% 

Confusion Matrix:
   True Positives: 8
   False Positives: 1
   True Negatives: 6
   False Negatives: 6
  True Postive Rate: 57.14%
  True Negative Rate: 85.71%


The model had 14 correct out of 21. Resulted in accuracy of: 66.67% 

Confusion Matrix:
   True Positives: 7
   False Positives: 4
   True Negatives: 7
   False Negatives: 3
  True Postive Rate: 70.00%
  True Negative Rate: 63.64%


The model had 18 correct out of 21. Resulted in accuracy of: 85.71% 

Confusion Matrix:
   True Positives: 12
   False Positives: 0
   True Negatives: 6
   False Negatives: 3
  True Postive Rate: 80.00%
  True Negative Rate: 100.00%


The model had 17 correct out of 21. Resulted in accuracy of: 80.95% 

Confusion Matrix:
   True Positives: 11
   False Positives: 2
   True Negatives: 6
   False Negatives: 2
  True Postive Rate: 84.62%
  True Negative Rate: 75.00%


The model had 18 correct out 

In [0]:
"""
Logistic regression Run of PARKINSONS
I pick thoses specific hyperparameters because through some experimentation I found to deliver the best performance
"""
startTestString="""
-----------------------------------------------------------------------------------------------
Logistic regression test run for the PARKINSONS Data set (entire and selected feature Dataset)
-----------------------------------------------------------------------------------------------
"""
print(startTestString)

experimentLearningRatePark=0.1
experimentNumItterations=140
#ALL FEATURES
parkAllAccLog = []
parkSelectAccLog= []
for _ in range(5):
  parkinsonsKFold = KFoldValidation(parkinsonDF, 10, 'status') #K fold with all features
  parkinsonsKFold.kFold_Log_fit(experimentLearningRatePark,experimentNumItterations)
  parkinsonsAvgAccLog = parkinsonsKFold.kFold_Log_predict()
  parkAllAccLog.append(format(parkinsonsAvgAccLog,".2%"))
#SELECT FEATURES
for _ in range(5):
  parkinsonsKFoldFeat = KFoldValidation(featSelectPark, 10, 'status') #K fold with selected features
  parkinsonsKFoldFeat.kFold_Log_fit(experimentLearningRatePark,experimentNumItterations)
  parkinsonsFeatAvgAccLog = parkinsonsKFoldFeat.kFold_Log_predict()
  parkSelectAccLog.append(format(parkinsonsFeatAvgAccLog, ".2%"))



-----------------------------------------------------------------------------------------------
Logistic regression test run for the PARKINSONS Data set (entire and selected feature Dataset)
-----------------------------------------------------------------------------------------------



/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:22: RuntimeWarning: overflow encountered in exp


Summary for Logistic Regression:
The model had 16 correct out of 20. Resulted in accuracy of: 80.00% 

Confusion Matrix:
   True Positives: 16
   False Positives: 4
   True Negatives: 0
   False Negatives: 0
  True Postive Rate: 100.00%
  True Negative Rate: 0.00%


Summary for Logistic Regression:
The model had 15 correct out of 20. Resulted in accuracy of: 75.00% 

Confusion Matrix:
   True Positives: 15
   False Positives: 5
   True Negatives: 0
   False Negatives: 0
  True Postive Rate: 100.00%
  True Negative Rate: 0.00%


Summary for Logistic Regression:
The model had 14 correct out of 20. Resulted in accuracy of: 70.00% 

Confusion Matrix:
   True Positives: 14
   False Positives: 6
   True Negatives: 0
   False Negatives: 0
  True Postive Rate: 100.00%
  True Negative Rate: 0.00%


Summary for Logistic Regression:
The model had 6 correct out of 20. Resulted in accuracy of: 30.00% 

Confusion Matrix:
   True Positives: 0
   False Positives: 0
   True Negatives: 6
   False Negati

In [0]:
"""
Logistic regression Run of Sonar 
I pick thoses specific hyperparameters because through some experimentation I found to deliver the best performance
"""
startTestString="""
-----------------------------------------------------------------------------------------------
Logistic regression test run for the Sonar Data set (entire and selected feature Dataset)
-----------------------------------------------------------------------------------------------
"""
print(startTestString)

experimentLearningRate=0.01
experimentNumItterations=1400
#ALL FEATURES

sonarAllAccLog = []
sonarSelectAccLog = []
for _ in range(5):
  sonarKFold = KFoldValidation(sonarDF, 10, 'Feature 60') #K fold with all features
  sonarKFold.kFold_Log_fit(experimentLearningRate,experimentNumItterations)
  sonarAllAvgAccLog=sonarKFold.kFold_Log_predict()
  sonarAllAccLog.append(format(sonarAllAvgAccLog,".2%"))
#SELECT FEATURES
for _ in range(5):
  sonarKFoldFeat = KFoldValidation(featSelectSonar, 10, 'Feature 60') #K fold with selected features
  sonarKFoldFeat.kFold_Log_fit(experimentLearningRate,experimentNumItterations)
  sonarFeatAvgAccLog = sonarKFoldFeat.kFold_Log_predict()
  sonarSelectAccLog.append(format(sonarFeatAvgAccLog, ".2%"))




-----------------------------------------------------------------------------------------------
Logistic regression test run for the Sonar Data set (entire and selected feature Dataset)
-----------------------------------------------------------------------------------------------

Summary for Logistic Regression:
The model had 17 correct out of 21. Resulted in accuracy of: 80.95% 

Confusion Matrix:
   True Positives: 8
   False Positives: 1
   True Negatives: 9
   False Negatives: 3
  True Postive Rate: 72.73%
  True Negative Rate: 90.00%


Summary for Logistic Regression:
The model had 15 correct out of 21. Resulted in accuracy of: 71.43% 

Confusion Matrix:
   True Positives: 7
   False Positives: 5
   True Negatives: 8
   False Negatives: 1
  True Postive Rate: 87.50%
  True Negative Rate: 61.54%


Summary for Logistic Regression:
The model had 14 correct out of 21. Resulted in accuracy of: 66.67% 

Confusion Matrix:
   True Positives: 6
   False Positives: 6
   True Negatives: 8

# Results

In [0]:
"""
RESULTS OF LDA
  - 5 runs of a 10-Fold LDA Each
"""

print("Average Accuricies for 5 runs of a 10-Fold Cross Validation Analysis of LDA\n")
print("Parkinsons (All Features):    " + str(parkAllAcc))
print("Parkinsons (Select Features): " + str(parkSelectAcc))
print("Sonar (All Features):         " + str(sonarAllAcc))
print("Sonar (Select Features):      " + str(sonarSelectAcc))


logResults=f"""
Average Accuricies for 5 runs of a 10-Fold Cross Validation Analysis of Logistic Regression

Parkinsons (All Features):    {parkAllAccLog}
Parkinsons (Select Features): {parkSelectAccLog}
Sonar (All Features):         {sonarAllAccLog}
Sonar (Select Features):      {sonarSelectAccLog}
"""
print(logResults)

Average Accuricies for 5 runs of a 10-Fold Cross Validation Analysis of LDA

Parkinsons (All Features):    ['87.16%', '88.66%', '89.16%', '88.76%', '88.32%']
Parkinsons (Select Features): ['84.13%', '82.58%', '85.26%', '84.66%', '85.11%']
Sonar (All Features):         ['74.45%', '76.05%', '77.38%', '75.93%', '75.48%']
Sonar (Select Features):      ['77.81%', '77.43%', '76.50%', '75.90%', '76.07%']

Average Accuricies for 5 runs of a 10-Fold Cross Validation Analysis of Logistic Regression

Parkinsons (All Features):    ['64.84%', '67.26%', '52.76%', '57.39%', '60.42%']
Parkinsons (Select Features): ['69.39%', '60.61%', '52.92%', '75.50%', '71.26%']
Sonar (All Features):         ['76.07%', '76.45%', '74.93%', '76.38%', '79.40%']
Sonar (Select Features):      ['72.52%', '72.14%', '72.19%', '71.60%', '71.57%']

time: 5.46 ms


# Extras (Other tests that were run for the report)

In [0]:
"""
Fitting and predicting with LDA on each dataset with entire data (not k fold)
"""

#LDA on Parkinsons
print("LDA PARKINSONS FULL")
ldaParkinsons = LDA(myFormatter.xMatrix(parkinsonDF, 'status'), myFormatter.yMatrix(parkinsonDF, 'status'))
parkPredictions = ldaParkinsons.predict(ldaParkinsons.X)
#Accuracy Metrics on Parkinsons
pMetric = MLMetric(parkPredictions, ldaParkinsons.Y)
acc = pMetric.Accu_eval()


#LDA on Parkinsons FEAT SELECT
print("LDA PARKINSONS Feature Select")
ldaParkFeat = LDA(myFormatter.xMatrix(featSelectPark, 'status'), myFormatter.yMatrix(featSelectPark, 'status'))
parkFeatPredictions = ldaParkFeat.predict(ldaParkFeat.X)
#Accuracy Metrics on Parkinsons
pfMetric = MLMetric(parkFeatPredictions, ldaParkFeat.Y)
acc = pfMetric.Accu_eval()


#LDA on Sonar
print("LDA SONAR FULL")
ldaSonar = LDA(myFormatter.xMatrix(sonarDF, 'Feature 60'), myFormatter.yMatrix(sonarDF, 'Feature 60'))
sonarPredictions = ldaSonar.predict(ldaSonar.X)
#Accuracy Metrics on Sonar
sMetric = MLMetric(sonarPredictions, ldaSonar.Y)
acc = sMetric.Accu_eval()


#LDA on Sonar FEAT SELECT
print("LDA SONAR Feature Select")
ldaSonarFeat = LDA(myFormatter.xMatrix(featSelectSonar, 'Feature 60'), myFormatter.yMatrix(featSelectSonar, 'Feature 60'))
sonarFeatPredictions = ldaSonarFeat.predict(ldaSonarFeat.X)
#Accuracy Metrics on Sonar
sfMetric = MLMetric(sonarFeatPredictions, ldaSonarFeat.Y)
acc = sfMetric.Accu_eval()

LDA PARKINSONS FULL
The model had 179 correct out of 195. Resulted in accuracy of: 91.79% 

Confusion Matrix:
   True Positives: 144
   False Positives: 13
   True Negatives: 35
   False Negatives: 3
  True Postive Rate: 97.96%
  True Negative Rate: 72.92%


LDA PARKINSONS Feature Select
The model had 167 correct out of 195. Resulted in accuracy of: 85.64% 

Confusion Matrix:
   True Positives: 140
   False Positives: 21
   True Negatives: 27
   False Negatives: 7
  True Postive Rate: 95.24%
  True Negative Rate: 56.25%


LDA SONAR FULL
The model had 188 correct out of 208. Resulted in accuracy of: 90.38% 

Confusion Matrix:
   True Positives: 103
   False Positives: 12
   True Negatives: 85
   False Negatives: 8
  True Postive Rate: 92.79%
  True Negative Rate: 87.63%


LDA SONAR Feature Select
The model had 164 correct out of 208. Resulted in accuracy of: 78.85% 

Confusion Matrix:
   True Positives: 86
   False Positives: 19
   True Negatives: 78
   False Negatives: 25
  True Postiv

In [0]:
"""
Fitting and predicting with Logistic Regression on entire dataset
"""
experimentLearningRate=0.1
experimentLearningRatePark=0.01
experimentNumItterations=1400
#LR on Parkinsons
print("Logistic Regerssion PARKINSONS FULL")
logParkinsons = LogisticRegression(myFormatter.xMatrix(parkinsonDF, 'status'), myFormatter.yMatrix(parkinsonDF, 'status'),experimentLearningRatePark,experimentNumItterations)
logParkinsons.fit(experimentLearningRatePark,experimentNumItterations)
parkPredictions = logParkinsons.predict(logParkinsons.X)
#Accuracy Metrics on Parkinsons
pMetric = MLMetric(parkPredictions, logParkinsons.Y)
acc = pMetric.Accu_eval()


#LR on Parkinsons FEAT SELECT
print("Logistic Regerssion PARKINSONS Feature Select")
logParkFeat =  LogisticRegression(myFormatter.xMatrix(featSelectPark, 'status'), myFormatter.yMatrix(featSelectPark, 'status'),experimentLearningRatePark,experimentNumItterations)
logParkFeat.fit(experimentLearningRatePark,experimentNumItterations)
parkFeatPredictions = logParkFeat.predict(logParkFeat.X)
#Accuracy Metrics on Parkinsons
pfMetric = MLMetric(parkFeatPredictions, logParkFeat.Y)
acc = pfMetric.Accu_eval()


#LR on Sonar
print("Logistic Regerssion SONAR FULL")
logSonar = LogisticRegression(myFormatter.xMatrix(sonarDF, 'Feature 60'), myFormatter.yMatrix(sonarDF, 'Feature 60'),experimentLearningRate,experimentNumItterations)
logSonar.fit(experimentLearningRatePark,experimentNumItterations)
sonarPredictions = logSonar.predict(logSonar.X)

#Accuracy Metrics on Sonar
sMetric = MLMetric(sonarPredictions, logSonar.Y)
acc = sMetric.Accu_eval()


#LR on Sonar FEAT SELECT
print("Logistic Regerssion SONAR Feature Select")
logSonarFeat = LogisticRegression(myFormatter.xMatrix(featSelectSonar, 'Feature 60'), myFormatter.yMatrix(featSelectSonar, 'Feature 60'),experimentLearningRatePark,experimentNumItterations)
logSonarFeat.fit(experimentLearningRatePark,experimentNumItterations)
sonarFeatPredictions = logSonarFeat.predict(logSonarFeat.X)
#Accuracy Metrics on Sonar
sfMetric = MLMetric(sonarFeatPredictions, logSonarFeat.Y)
acc = sfMetric.Accu_eval()

Logistic Regerssion PARKINSONS FULL


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:22: RuntimeWarning: overflow encountered in exp


The model had 147 correct out of 195. Resulted in accuracy of: 75.38% 

Confusion Matrix:
   True Positives: 147
   False Positives: 48
   True Negatives: 0
   False Negatives: 0
  True Postive Rate: 100.00%
  True Negative Rate: 0.00%


Logistic Regerssion PARKINSONS Feature Select
The model had 147 correct out of 195. Resulted in accuracy of: 75.38% 

Confusion Matrix:
   True Positives: 147
   False Positives: 48
   True Negatives: 0
   False Negatives: 0
  True Postive Rate: 100.00%
  True Negative Rate: 0.00%


Logistic Regerssion SONAR FULL
The model had 179 correct out of 208. Resulted in accuracy of: 86.06% 

Confusion Matrix:
   True Positives: 98
   False Positives: 16
   True Negatives: 81
   False Negatives: 13
  True Postive Rate: 88.29%
  True Negative Rate: 83.51%


Logistic Regerssion SONAR Feature Select
The model had 157 correct out of 208. Resulted in accuracy of: 75.48% 

Confusion Matrix:
   True Positives: 88
   False Positives: 28
   True Negatives: 69
   False N